### info

해당 주피터 노트북은 inf2.24xlarge 에서 neox-20b를 이용하여 테스트 (23.10.17) 되었으며 <br>
관리자 권한과 신뢰 관계가 설정이 된 IAM 권한이 있어야 함. <br>
EC2에도 설정한 IAM 권한이 부여되어있는 상태이여야함. <br>
사용된 AMI:ami-0c0925f65f99da6d0 (버지니아 북부) <br><br>
-----------------------------------------<br>
`./model` 폴더에는 inf2인증 및 sagemaker 서빙을 위한 파일들이 위치하고 있음 <br>
ex) 검증과정 회피를 위한 분기 처리 등... <br>
API 응답 관련 부분의 경우 `./model/scoring_logic.py` 수정 필요 <br>
모델 변경시 컴파일을 진행하는 코드인 `./neoxgpt_20b_compile.py` 코드 수정 필요 <br>



### AWS config

In [1]:
!mkdir ~/.aws
!echo "[default]" > ~/.aws/config 
!echo "region = us-east-1">> ~/.aws/config 

### 라이브러리 설치

In [2]:
!pip install bitsandbytes>=0.39.0
!pip install sentencepiece
!pip install transformers-neuronx==0.7.84 --extra-index-url=https://pip.repos.neuron.amazonaws.com
!pip install boto3
!pip install sagemaker
!pip install docker 

Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 8.0 MB/s eta 0:00:000:00:01
Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com, https://pip.repos.neuron.amazonaws.com
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.3/150.3 kB 2.1 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 258.1/258.1 kB 5.7 MB/s eta 0:00:00a 0:00:01
Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com
Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 902.8/902.8 kB 5.3 MB/s eta 0:00:0000:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.5/57.5 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.1/82.1 kB 1.5 MB/s eta 0:00:000:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.6/132.6 kB 18.5 MB/s

### inf2 빌드, 데이터 이동

In [3]:
!python neoxgpt_20b_compile.py
!mv gpt-neox-20b ./model
!mv neox_20b_f16.pkl ./model
!sudo chmod -R 777 ./model

*	*	 Origin Load model	*	*
Loading checkpoint shards: 100%|████████████████| 46/46 [00:05<00:00,  7.86it/s]
Load model time :: 192.10230159759521
*	*	 save pretrained split	*	*
save pretrained split time :: 104.9177737236023
*	*	 Compile model	*	*
/opt/aws_neuron_venv_pytorch/lib/python3.8/site-packages/transformers_neuronx/gptneox/model.py:41: UserWarning: hidden_act="gelu_fast" ignored in favor of hidden_act="gelu_new"
  warnings.warn(f'hidden_act="{self.config.activation_function}" ignored in favor of hidden_act="gelu_new"')
2023-10-17 06:00:50.000102:  25887  INFO ||NEURON_CACHE||: Compile cache path: /var/tmp/neuron-compile-cache
2023-10-17 06:00:50.000114:  25887  INFO ||NEURON_CC_WRAPPER||: Call compiler with cmd: ['neuronx-cc', '--target=trn1', 'compile', '--framework', 'XLA', '/tmp/neuroncc_compile_workdir/f79e932f-1032-44f7-8504-58263b219983/model.MODULE_6089c2d9833b83667058+a1fe2fe5.hlo.pb', '--output', '/tmp/neuroncc_compile_workdir/f79e932f-1032-44f7-8504-58263b219983/mode

### boto3 및 필요 서비스, 정보 로드

In [4]:
import base64
import boto3
import docker
import json
import pandas as pd
import requests
import sagemaker as sage
from sagemaker import get_execution_role, ModelPackage
import socket
import time
from urllib.parse import urlparse

session = sage.Session()
model_bucket = "neuron-model-4"
s3_bucket = session.default_bucket()
region = session.boto_region_name
account_id = boto3.client("sts").get_caller_identity().get("Account")

# IAM설정이 되어 있어야함, 설정한 IAM 정보로 변경
role = "arn:aws:iam::994376801009:role/admin"

sagemaker = boto3.client("sagemaker")
s3_client = session.boto_session.client("s3")
ecr = boto3.client("ecr")
sm_runtime = boto3.client("sagemaker-runtime")

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ubuntu/.config/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ubuntu/.config/sagemaker/config.yaml


In [5]:
model_name = "neox-20b-4"

### S3 파일 제작, 업로드

In [6]:
!tar -zcvf model_name.tar.gz ./model

./model/
./model/serve
./model/program.py
./model/neox_20b_f16.pkl
./model/gpt-neox-20b/
./model/gpt-neox-20b/generation_config.json
./model/gpt-neox-20b/config.json
./model/gpt-neox-20b/pytorch_model.bin/
./model/gpt-neox-20b/pytorch_model.bin/p439.gpt_neox.layers.33.mlp.dense_h_to_4h.weight
./model/gpt-neox-20b/pytorch_model.bin/p154.gpt_neox.layers.11.mlp.dense_h_to_4h.bias
./model/gpt-neox-20b/pytorch_model.bin/p376.gpt_neox.layers.28.mlp.dense_4h_to_h.weight
./model/gpt-neox-20b/pytorch_model.bin/p122.gpt_neox.layers.9.attention.rotary_emb.inv_freq
./model/gpt-neox-20b/pytorch_model.bin/p23.gpt_neox.layers.1.mlp.dense_h_to_4h.weight
./model/gpt-neox-20b/pytorch_model.bin/p319.gpt_neox.layers.24.attention.query_key_value.bias
./model/gpt-neox-20b/pytorch_model.bin/p218.gpt_neox.layers.16.mlp.dense_h_to_4h.weight
./model/gpt-neox-20b/pytorch_model.bin/p79.gpt_neox.layers.6.input_layernorm.weight
./model/gpt-neox-20b/pytorch_model.bin/p565.gpt_neox.layers.43.attention.query_key_value

In [7]:
from botocore.exceptions import ClientError
s3 = boto3.resource('s3')
def create_bucket(bucket_name, region='us-east-1'):
    try:
        if region == 'us-east-1':
            s3.create_bucket(Bucket=bucket_name)
        else:
            s3.create_bucket(Bucket=bucket_name, CreateBucketConfiguration={'LocationConstraint': region})
    except ClientError as e:
        if e.response['Error']['Code'] == 'BucketAlreadyOwnedByYou':
            print(f"{bucket_name} already exists.")
        else:
            print("An error occurred while creating the bucket.")
            raise e
    else:
        print(f"{bucket_name} created.")

create_bucket(model_bucket, region=region)

neuron-model-4 created.


In [8]:
object_name = model_name+"-4.tar.gz"
def upload_bucket(file_name, bucket, object):
    try:
        resposne = s3_client.upload_file(file_name, bucket, object)
    except ClientError as e:
        logging.error(e)
        return False
    return True
    
upload_bucket("model_name.tar.gz", model_bucket, object_name)

True

### 도커 build

In [9]:
!aws ecr get-login-password --region us-west-2 | docker login --username AWS --password-stdin 763104351884.dkr.ecr.us-west-2.amazonaws.com

WARNING! Your password will be stored unencrypted in /home/ubuntu/.docker/config.json.
Configure a credential helper to remove this warning. See
https://docs.docker.com/engine/reference/commandline/login/#credentials-store

Login Succeeded


In [10]:
!docker pull 763104351884.dkr.ecr.us-west-2.amazonaws.com/pytorch-inference-neuronx:1.13.1-neuronx-py310-sdk2.14.1-ubuntu20.04

1.13.1-neuronx-py310-sdk2.14.1-ubuntu20.04: Pulling from pytorch-inference-neuronx

490126ef: Pulling fs layer 
2d39b0e3: Pulling fs layer 
c149daad: Pulling fs layer 
d542f6a1: Pulling fs layer 
a846ec65: Pulling fs layer 
d905f9b2: Pulling fs layer 
fa6d4f9a: Pulling fs layer 
f8f2df03: Pulling fs layer 
53194264: Pulling fs layer 
bd2643f1: Pulling fs layer 
b7ca1802: Pulling fs layer 
dfd859da: Pulling fs layer 
f8f2df03: Waiting fs layer 
846ec65: Waiting fs layer 
905f9b2: Waiting fs layer 
0dfbef52: Pulling fs layer 
53194264: Waiting fs layer 
d17af7b7: Pulling fs layer 
fe0e7e07: Pulling fs layer 
a8542ad0: Pulling fs layer 
e23739d5: Pulling fs layer 
ead242bd: Pulling fs layer 
Digest: sha256:96b03a837a362f9f11bc8f49817bcfae8b17daa1527950c89b3e2def7fa3f5792K
Status: Downloaded newer image for 763104351884.dkr.ecr.us-west-2.amazonaws.com/pytorch-inference-neuronx:1.13.1-neuronx-py310-sdk2.14.1-ubuntu20.04
763104351884.dkr.ecr.us-west-2.amazonaws.com/pytorch-inference-neuronx:

In [11]:
docker_client = docker.from_env()
image, build_logs = docker_client.images.build(path="./docker", tag=model_name)

In [12]:
docker_image_arn = f"{account_id}.dkr.ecr.{region}.amazonaws.com/{model_name}"
docker_image_arn

'994376801009.dkr.ecr.us-east-1.amazonaws.com/neox-20b-4'

### 도커 ECR 업로드

In [13]:
repo_exists = model_name in [
    repo["repositoryName"] for repo in ecr.describe_repositories().get("repositories")
]

if not repo_exists:
    ecr.create_repository(repositoryName=model_name)

In [14]:
ecr_auth_data = ecr.get_authorization_token()["authorizationData"][0]
username, password = (
    base64.b64decode(ecr_auth_data["authorizationToken"]).decode("utf-8").split(":")
)

docker_client.api.tag(model_name, docker_image_arn, tag="latest")
status = docker_client.api.push(
    docker_image_arn,
    tag="latest",
    auth_config={"username": username, "password": password},
)

### 검증용 데이터 업로드

In [15]:
csv_line = "what is aws"

with open("input.txt", "w") as f:
    f.write(csv_line)

s3_client.put_object(Bucket=s3_bucket, Key="validation-input-csv/input.txt", Body=csv_line)

{'ResponseMetadata': {'RequestId': 'KWKP82C2NPX2TKXR',
  'HostId': 'ZFp44NI1DkO5X4APT54iut6DRKfM3zTi69M8eA6RkV9tX1N4jDjD6qtDnMBCltQj0wcfQuSeR3s=',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': 'ZFp44NI1DkO5X4APT54iut6DRKfM3zTi69M8eA6RkV9tX1N4jDjD6qtDnMBCltQj0wcfQuSeR3s=',
   'x-amz-request-id': 'KWKP82C2NPX2TKXR',
   'date': 'Tue, 17 Oct 2023 06:54:53 GMT',
   'x-amz-server-side-encryption': 'AES256',
   'etag': '"5fa53936f2be5171a33cb765d5a7f3a0"',
   'server': 'AmazonS3',
   'content-length': '0'},
  'RetryAttempts': 0},
 'ETag': '"5fa53936f2be5171a33cb765d5a7f3a0"',
 'ServerSideEncryption': 'AES256'}

In [16]:
validation_file_name = "input.txt"
validation_input_path = f"s3://{s3_bucket}/validation-input-csv/"
validation_output_path = f"s3://{s3_bucket}/validation-output-csv/"

### 모델 패키지 생성 및 검증

In [17]:
model_description = "This model accepts petal length, petal width, sepal length, sepal width and predicts whether flower is of type setosa, versicolor, or virginica"

supported_content_types = ["text/plain"]
supported_response_MIME_types = [
    "text/plain",
]

In [19]:
model_package = sagemaker.create_model_package(
    ModelPackageName='neo20b-neuron-4',
    ModelPackageDescription="This model is neox20b complied to use in inf2",
    InferenceSpecification={
        "Containers": [
            {
                "Image": "994376801009.dkr.ecr.us-east-1.amazonaws.com/neox-20b-4:latest",
                "ModelDataUrl": "s3://neuron-model-4/neox-20b-4-4.tar.gz",
            }
        ],
        "SupportedTransformInstanceTypes": ["ml.g4dn.12xlarge"],
        "SupportedRealtimeInferenceInstanceTypes": ["ml.inf2.24xlarge"],
        "SupportedContentTypes": ["text/plain"],
        "SupportedResponseMIMETypes": ["text/plain"],
    },
    CertifyForMarketplace=True,  # Make sure to set this to True for Marketplace models!
    ValidationSpecification={
        "ValidationRole": role,
        "ValidationProfiles": [
            {
                "ProfileName": "Validation-test",
                "TransformJobDefinition": {
                    "BatchStrategy": "SingleRecord",
                    "TransformInput": {
                        "DataSource": {
                            "S3DataSource": {
                                "S3DataType": "S3Prefix",
                                "S3Uri": validation_input_path,
                            }
                        },
                        "ContentType": "text/plain",
                    },
                    "TransformOutput": {
                        "S3OutputPath": validation_output_path,
                    },
                    "TransformResources": {
                        "InstanceType": "ml.g4dn.12xlarge",
                        "InstanceCount": 1,
                    },
                },
            },
        ],
    },
)

In [20]:
session.wait_for_model_package(model_package_name='neo20b-neuron-4')
# neox20b 기준 약20분 소요

....................................................................................................................................................................................................................................


{'ModelPackageName': 'neo20b-neuron-4',
 'ModelPackageArn': 'arn:aws:sagemaker:us-east-1:994376801009:model-package/neo20b-neuron-4',
 'ModelPackageDescription': 'This model is neox20b complied to use in inf2',
 'CreationTime': datetime.datetime(2023, 10, 17, 6, 56, 6, 770000, tzinfo=tzlocal()),
 'InferenceSpecification': {'Containers': [{'Image': '994376801009.dkr.ecr.us-east-1.amazonaws.com/neox-20b-4:latest',
    'ImageDigest': 'sha256:a404831526bfeb165f06b8a845996cb6f38279c8260f684e32619dac88b45dd3',
    'ModelDataUrl': 's3://neuron-model-4/neox-20b-4-4.tar.gz'}],
  'SupportedTransformInstanceTypes': ['ml.g4dn.12xlarge'],
  'SupportedRealtimeInferenceInstanceTypes': ['ml.inf2.24xlarge'],
  'SupportedContentTypes': ['text/plain'],
  'SupportedResponseMIMETypes': ['text/plain']},
 'ValidationSpecification': {'ValidationRole': 'arn:aws:iam::994376801009:role/admin',
  'ValidationProfiles': [{'ProfileName': 'Validation-test',
    'TransformJobDefinition': {'BatchStrategy': 'SingleRecor

### 모델 패키지 이용

In [21]:
model = ModelPackage(
    role=role,
    model_package_arn=model_package["ModelPackageArn"],
    sagemaker_session=session,
)

In [22]:
model.deploy(
    initial_instance_count=1,
    instance_type="ml.inf2.24xlarge",
    endpoint_name="neox20b-neuron",
    volume_size=500,
    model_data_download_timeout=1800,
    container_startup_health_check_timeout=3600
)
model.endpoint_name

Your model is not compiled. Please compile your model before using Inferentia.


-------------------!

'neox20b-neuron'

### 생성된 endpoint 호출

In [23]:
response = sm_runtime.invoke_endpoint(
    EndpointName=model.endpoint_name,
    ContentType="text/plain",
    Accept="application/json",
    Body="what is aws",
)

br = response["Body"]
print(br)
print(br.read())

b'what is aws s3 storage class and what does it do\n\nI created the following bucket in aws console.  Is there any limit on the number of files in a bucket?  How many files can be uploaded on Amazon cloud? What does storage class option do in the S3 bucket?\n\nCan someone clarify these for me?\n\nA:\n\n"Storage class" (or just "class") is just a predefined storage type. For example, you can set the class to "Standard" and set the storage cost to be $0.0001. Then, your limit will be $1.000/million/year. \nThe Storage class will not effect the number of objects that you can create in your bucket, however, you can also limit the number of objects by using the ACL. For example, you can create two objects, bucket-1 and bucket-2. And set the ACL to Bucket-1 be set to: private, only bucket-1 and bucket-2.\nbucket-1:\n  private\n  only bucket-1, bucket-2\n\nbucket-2:\n  private\n  only bucket-1, bucket-2\n\nFor your second question, you can upload as many objects as you want. The upload limit 

### 사용된 endpoint 제거

In [24]:
model.sagemaker_session.delete_endpoint(model.endpoint_name)
model.sagemaker_session.delete_endpoint_config(model.endpoint_name)